In [ ]:
import requests
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# API URLs
Views_URL = "https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
likes_URL = "https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
insp_URL = "https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
rated_URL = "https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
post_URL = "https://api.socialverseapp.com/posts/summary/get?page=1&page_size=1000"
user_URL = "https://api.socialverseapp.com/users/get_all?page=1&page_size=1000"
HEADERS = {
    "Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
}




def get_data(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        posts = data.get("posts", [])
        return pd.DataFrame(posts)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")
        return pd.DataFrame()

# Fetch data
view = get_data(Views_URL)   # id, post_id, user_id, viewed_at
liked = get_data(likes_URL)    # id, post_id, user_id, liked_at
inspired = get_data(insp_URL)   # id, post_id, user_id, inspired_at
rated = get_data(rated_URL)   # id, post_id, user_id, rating_percent, rated_at
posts = get_data(post_URL)
users = get_data(user_URL)

# Merge all interaction data into one DataFrame
view['action'] = 'viewed'
liked['action'] = 'liked'
inspired['action'] = 'inspired'
rated['action'] = 'rated'

# Combine all user interactions into a single DataFrame
interaction_data = pd.concat([view[['user_id', 'post_id', 'action']],
                              liked[['user_id', 'post_id', 'action']],
                              inspired[['user_id', 'post_id', 'action']],
                              rated[['user_id', 'post_id', 'action']]])
interaction_matrix = pd.pivot_table(interaction_data, index='user_id', columns='post_id', values='action', aggfunc=lambda x: 1, fill_value=0)
print(interaction_matrix.head())




# Step 1: Collaborative Filtering - Calculate User Similarity using Cosine Similarity
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)





# Step 2: Recommend posts based on user similarity
def get_collaborative_recommendations(user_id, interaction_matrix, user_similarity_df, top_n=5):
    user_ratings = interaction_matrix.loc[user_id]
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    recommended_posts = []
    for similar_user in similar_users.index:
        similar_user_ratings = interaction_matrix.loc[similar_user]
        not_rated_posts = similar_user_ratings[similar_user_ratings > 0].index.difference(user_ratings[user_ratings > 0].index)
        recommended_posts.extend(not_rated_posts)
    recommended_posts = list(set(recommended_posts))[:top_n]
    return posts[posts['id'].isin(recommended_posts)]




# Example: Get collaborative recommendations for user 1
collab_recs = get_collaborative_recommendations(1, interaction_matrix, user_similarity_df)
print("\nCollaborative Filtering Recommendations for User 1:")
print(collab_recs[['title', 'category']])






# Step 3: Content-Based Filtering (Recommend posts based on category)
def get_content_recommendations(post_id, posts):
    post_category = posts.loc[posts['id'] == post_id, 'category'].values
    similar_posts = posts[posts['category'] == post_category]
    return similar_posts




# Example: Get content-based recommendations for post 1
content_recs = get_content_recommendations(1, posts)
print("\nContent-Based Recommendations for Post 1:")
print(content_recs[['title', 'category']])




# Step 4: Hybrid Model - Combine collaborative and content-based recommendations
def hybrid_recommendations(user_id, post_id, interaction_matrix, user_similarity_df, posts, top_n=5):
    # Get collaborative recommendations
    collab_recs = get_collaborative_recommendations(user_id, interaction_matrix, user_similarity_df, top_n)

    # Get content-based recommendations
    content_recs = get_content_recommendations(post_id, posts)

    # Combine both recommendations and remove duplicates
    combined_recs = pd.concat([collab_recs, content_recs]).drop_duplicates()
    return combined_recs



# Example: Get hybrid recommendations for user 1 and post 1
hybrid_recs = hybrid_recommendations(1, 1, interaction_matrix, user_similarity_df, posts)
print("\nHybrid Recommendations for User 1 and Post 1:")
print(hybrid_recs[['title', 'category']])
